In [ ]:
!pip -q install -U langgraph langchain-openai langchain-core

import os
import json
from getpass import getpass
from typing import TypedDict

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY (hidden): ")

MODEL = os.environ.get("OPENAI_MODEL", "gpt-4o-mini")
llm = ChatOpenAI(model=MODEL, temperature=0.2)

In [ ]:
class AnemoiState(TypedDict):
    task: str
    max_rounds: int
    round: int
    draft: str
    critique: str
    agreed: bool
    final: str
    trace: bool

In [ ]:
DRAFTER_SYSTEM = """You are Agent A (Drafter) in a peer-to-peer loop.
You write a high-quality solution to the user's task.
If you receive critique, you revise decisively and incorporate it.
Return only the improved draft text."""

def drafter_node(state: AnemoiState) -> AnemoiState:
    task = state["task"]
    critique = state.get("critique", "").strip()
    r = state.get("round", 0) + 1

    if critique:
        user_msg = f"""TASK:
{task}

CRITIQUE:
{critique}

Revise the draft."""
    else:
        user_msg = f"""TASK:
{task}

Write the first draft."""

    draft = llm.invoke(
        [
            {"role": "system", "content": DRAFTER_SYSTEM},
            {"role": "user", "content": user_msg},
        ]
    ).content.strip()

    if state.get("trace", False):
        print(f"\n--- Drafter Round {r} ---\n{draft}\n")

    return {**state, "round": r, "draft": draft, "agreed": False}

In [ ]:
CRITIC_SYSTEM = """You are Agent B (Critic).
Return strict JSON:
{"agree": true/false, "critique": "..."}"""

def critic_node(state: AnemoiState) -> AnemoiState:
    task = state["task"]
    draft = state.get("draft", "")

    raw = llm.invoke(
        [
            {"role": "system", "content": CRITIC_SYSTEM},
            {
                "role": "user",
                "content": f"TASK:\n{task}\n\nDRAFT:\n{draft}",
            },
        ]
    ).content.strip()

    cleaned = raw.strip("```").replace("json", "").strip()

    try:
        data = json.loads(cleaned)
        agree = bool(data.get("agree", False))
        critique = str(data.get("critique", "")).strip()
    except Exception:
        agree = False
        critique = raw

    if state.get("trace", False):
        print(f"--- Critic Decision ---\nAGREE: {agree}\n{critique}\n")

    final = draft if agree else state.get("final", "")
    return {**state, "agreed": agree, "critique": critique, "final": final}

In [3]:
def continue_or_end(state: AnemoiState) -> str:
    if state.get("agreed", False):
        return "end"
    if state.get("round", 0) >= state.get("max_rounds", 3):
        return "force_ship"
    return "loop"

def force_ship_node(state: AnemoiState) -> AnemoiState:
    return {**state, "final": state.get("final") or state.get("draft", "")}

graph = StateGraph(AnemoiState)
graph.add_node("drafter", drafter_node)
graph.add_node("critic", critic_node)
graph.add_node("force_ship", force_ship_node)

graph.set_entry_point("drafter")
graph.add_edge("drafter", "critic")
graph.add_conditional_edges(
    "critic",
    continue_or_end,
    {"loop": "drafter", "force_ship": "force_ship", "end": END},
)
graph.add_edge("force_ship", END)

anemoi_critic_loop = graph.compile()

demo_task = """Explain the Anemoi semi-centralized agent pattern and why peer-to-peer critic loops reduce bottlenecks."""

result = anemoi_critic_loop.invoke(
    {
        "task": demo_task,
        "max_rounds": 3,
        "round": 0,
        "draft": "",
        "critique": "",
        "agreed": False,
        "final": "",
        "trace": False,
    }
)

print("\n====================")
print("✅ FINAL OUTPUT")
print("====================\n")
print(result["final"])


--- Agent A (Round 1) Draft ---
The "Anemoi" semi-centralized pattern for agentic systems is designed to enhance efficiency and responsiveness in collaborative environments by reducing bottlenecks typically associated with traditional manager-worker hierarchies. In conventional systems, decision-making often becomes a slow process as information must flow through multiple layers of management. This can lead to delays in action and a lack of agility in responding to changes. The Anemoi pattern addresses this by distributing authority among agents while maintaining a central coordinating mechanism, allowing for faster decision-making and implementation.

In the Anemoi framework, each agent operates with a degree of autonomy, empowered to make decisions relevant to their specific tasks. This decentralization minimizes the need for constant managerial oversight, thereby reducing the bottlenecks that can arise when all decisions must be escalated to a manager. By allowing agents to act ind